In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D

from glob import glob
import time
import os

from sklearn.svm import LinearSVC 
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

import pipeline as pl

# Reload custum module everytime
%load_ext autoreload
%aimport pipeline
%autoreload 1


# Load Training and Test Datasets

In [ ]:
mldata_path = '/home/timo/Documents/mldata/project_5/'    

In [ ]:
dataset = pl.get_image_files(mldata_path)
print(len(dataset['files']))
print(len(dataset['labels']))

In [ ]:
for n in range(0, 3):
    plt.imshow(mpimg.imread(dataset['files'][n]))
    print(dataset['labels'][n])
    plt.show()

In [ ]:
img = mpimg.imread(dataset['files'][0])

colorspace = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = 0 # Can be 0, 1, 2, or "ALL"

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

hog = pl.get_hog_features(gray, orient, pix_per_cell, cell_per_block, vis=True)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 3))
axes[0].imshow(img)
axes[1].imshow(hog[1], 'gray')
axes[2].plot(hog[0])
axes[3].hist(hog[0], bins=32, range=[0.025,0.2])
plt.show()

In [ ]:
nb_samples = 1000# min(len(image_files['vehicles']), len(image_files['non-vehicles']))
kw_extract = {'color_space':'RGB',
              'spatial_size':(16, 16),
              'hist_bins':32,
              'orient':9,              
              'pix_per_cell':8,
              'cell_per_block':2,
              'hog_channel':0,
              'spatial_feat':True,
              'hist_feat':True,
              'hog_feat':True}
kw_split   = {'test_size': 0.2}


X, y, f = pl.extract_and_split(dataset, kw_extract, kw_split, nb_samples)

In [ ]:
plt.figure(figsize=(20,3))
plt.hist(features.ravel(), bins=256)
plt.grid('on')
plt.show()

In [ ]:
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()

svc.fit(X['train'], y['train'])
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print("Test Accuracy of SVC = {}".format(svc.score(X['test'], y['test']), 4))
# Check the prediction time for a single sample
t=time.time()

In [ ]:
n_predict = 10
print('My SVC predicts: ', svc.predict(X['test'][0:n_predict]))
print('For these',n_predict, 'labels: ', y['test'][0:n_predict].T)
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

In [ ]:
n = 6
plt.imshow(mpimg.imread(f['test'][n]))
print(y['test'][n])
plt.show()